In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import os

In [2]:
import sys
from pathlib import Path

# Add project root (parent of the optimization folder) to sys.path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [3]:
THEATER_NAME = "AMC Boston Common 19"
NUM_SCREENS = 19
SCREENS = [f"Screen_{i}" for i in range(1, NUM_SCREENS+1)]

In [4]:
DAYS = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
OPENING_TIME = dt.time(11,0)
CLOSING_TIME=dt.time(23,0)
TICKET_PRICE = 11.31

TICKET_PRICES = [TICKET_PRICE]*len(DAYS)
today = dt.date.today()
open_dt = dt.datetime.combine(today, OPENING_TIME)
close_dt = dt.datetime.combine(today, CLOSING_TIME)
delta = close_dt - open_dt


OPERATING_MIN_PER_DAY = {
    day: delta.total_seconds()/60 for day in DAYS  # same for all days for now
}

SCREEN_CAPACITIES = {s: 210 
for s in SCREENS[:19]}
SCREEN_CAPACITIES["Screen_19"] = 600

BUFFER_MIN = 15

MAX_SHOWINGS_PER_MOVIE_PER_DAY = 5

# # Synthetic movie Data 
# movies_df = pd.DataFrame({
#     "movie_id": ["M1", "M2", "M3", "M4"],
#     "title": ["Blockbuster Action", "Indie Drama", "Family Animation", "Horror Thriller"],
#     "runtime_min": [130, 110, 95, 100],
#     "ticket_price": [15.0, 14.0, 13.0, 13.0]
# })

In [5]:
from tmdb_api_calling.utils import get_movies_out_now, get_movies_from_url

In [6]:
tmdb_key = os.getenv("TMDB_API_KEY")
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_key}"
}

In [20]:
# CURRENT_MOVIES = get_movies_out_now(headers)
movies_df = pd.read_csv("../data/cleaned/final_merged_dataset_with_genres.csv")

In [49]:
current = movies_df[movies_df['release_date'] > '2025-10-01']

In [53]:
current

,ticker,date,title,distributor,gross,percent_yd,percent_lw,theaters,per_theater,total_gross,...,history,horror,music,mystery,romance,science_fiction,thriller,tv_movie,war,western
267,Private,2025-11-06,Anniversary,Roadside Att…,23223,-0.50,-0.43,809.0,29.0,530320,...,0,0,0,0,0,0,0,0,0,0
475,CMCSA,2025-11-10,Black Phone 2,Universal,551845,-0.59,-0.27,2943.0,188.0,70558650,...,0,1,0,0,0,0,1,0,0,0
1306,LGF.A,2025-11-10,Good Fortune,Lionsgate,70339,-0.53,-0.64,746.0,94.0,16188087,...,0,0,0,0,0,0,0,0,0,0
1773,Private,2025-11-06,Kiss of the Spider Woman,Roadside Att…,48,-0.83,-0.97,11.0,4.0,1623221,...,0,0,0,0,0,0,0,0,0,0
1815,Private,2025-11-06,Last Days,Vertical Ent…,664,-0.46,-0.84,74.0,9.0,219915,...,0,0,0,0,0,0,0,0,0,0
2496,Private,2025-10-19,Re-Election,Picturehouse,2215,-0.40,1.01,2.0,1108.0,19222,...,0,0,0,0,0,0,0,0,0,0
2519,PARA,2025-11-10,Regretting You,Paramount Pi…,811523,-0.50,-0.23,3196.0,254.0,38936925,...,0,0,0,0,1,0,0,0,0,0
2585,PARA,2025-11-10,Roofman,Paramount Pi…,52015,-0.49,-0.61,545.0,95.0,22376641,...,0,0,0,0,0,0,0,0,0,0
2722,Private,2025-11-09,Shelby Oaks,Neon,25000,-0.19,-0.87,200.0,125.0,4400918,...,0,1,0,1,0,0,1,0,0,0
2836,SONY,2025-11-10,Soul on Fire,Sony Pictures,9079,-0.69,-0.55,275.0,33.0,7349877,...,0,0,0,0,0,0,0,0,0,0


In [54]:
current['weeks_in_release'] = current['weeks_in_release'].apply(lambda x: x if x > 0 else 1)

/var/folders/mq/p952ryqs1gv79cnvz9f05slr0000gn/T/ipykernel_29879/3178696047.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current['weeks_in_release'] = current['weeks_in_release'].apply(lambda x: x if x > 0 else 1)


In [55]:
current['weekly_gross_adjusted_per_theater'] = current['gross_per_theater_adjusted_2024'] / current['weeks_in_release']

/var/folders/mq/p952ryqs1gv79cnvz9f05slr0000gn/T/ipykernel_29879/66180017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current['weekly_gross_adjusted_per_theater'] = current['gross_per_theater_adjusted_2024'] / current['weeks_in_release']


In [56]:
current['weekly_demand_per_theater'] = current.weekly_gross_adjusted_per_theater / TICKET_PRICE

/var/folders/mq/p952ryqs1gv79cnvz9f05slr0000gn/T/ipykernel_29879/473287358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current['weekly_demand_per_theater'] = current.weekly_gross_adjusted_per_theater / TICKET_PRICE


In [57]:
current[[
    'title_key', 'date', 'gross', 'average_gross','popularity', 'weeks_in_release','is_weekend', 'total_gross_adjusted_2024', 'gross_per_theater_adjusted_2024','weekly_demand_per_theater'
]].head(10)

,title_key,date,gross,average_gross,popularity,weeks_in_release,is_weekend,total_gross_adjusted_2024,gross_per_theater_adjusted_2024,weekly_demand_per_theater
267,anniversary,2025-11-06,23223,2.322300e+04,0.0773,1,0,530320.0,655.525340,57.959800
475,black phone 2,2025-11-10,551845,2.762397e+06,166.8039,3,0,70558650.0,23975.076453,706.604081
1306,good fortune,2025-11-10,70339,6.715468e+05,0.0604,3,0,16188087.0,21699.848525,639.547555
1773,kiss of the spider woman,2025-11-06,48,2.047375e+04,3.6080,4,0,1623221.0,147565.545455,3261.837875
1815,last days,2025-11-06,664,2.519825e+04,2.1153,2,0,219915.0,2971.824324,131.380386
2496,re-election,2025-10-19,2215,2.462250e+03,1.4877,1,1,19222.0,9611.000000,849.778957
2519,regretting you,2025-11-10,811523,2.152311e+06,65.7274,2,0,38936925.0,12183.017835,538.594953
2585,roofman,2025-11-10,52015,7.018551e+05,15.9921,4,0,22376641.0,41058.056881,907.560939
2722,shelby oaks,2025-11-09,25000,2.614580e+05,14.1275,2,1,4400918.0,22004.590000,972.793546
2836,soul on fire,2025-11-10,9079,2.453708e+05,2.7327,4,0,7349877.0,26726.825455,590.778635


# Optimization Formulation

1. Parameters

- $A_{i,d}$: expected attendance for movie i per day d

- $P_d$: average ticket price for movie on day d
- $D_i$: duration of movie i in minutes
- $B$: buffer time between shows
- $C_j$: capacity of screen j
- $H_{j,d}$: hours screen j operates on day d 
- $K$: maximum times a movie can be shown on a given day


2. Decision Variables:

- $s_{i,d}$: { 0, 1 }: whether movie i is scheduled on day d
- $x_{i, d, j}$: number of times we show movie i on screen j on a given day d 
- $r_{i, d}$: realized tickets served for movie i on day d, where $r_i,d=\min(j  x_i, d,j C_j,  A_i,d) $

3. Objective function: Maximize daily revenue:

$$max (\sum_ {i, d,j}x_{i, d, j}\times P_d r_{i,d}   ) $$


4. Constraints
- Total screen time: $\sum_{j} x_{i,d,j} (D_i+B) \leq H_{d,j} \forall d,j$
- Tickets served won’t exceed demand:  $r_i,d\leq \sum_{j} x_i, d,j C_j$
- Can’t sell more tickets than available: $r_i,d \leq A_i,d$  
- If not scheduled, no showings:  $\sum_{j} x_i, d,j \leq M*s_i,d$
- Each movie shown at most K times: $\sum_{j} x_i, d,j \leq K  ∀ i,d$
- Minimum number of genres each day: [TODO]

### Preprocessing

In [58]:
# Fixed: Create proper demand structure with (movie_id, day) tuples
# and distribute weekly demand across days

runtimes = {}
demand = {}
movie_ids = []

# Daily demand distribution (weekends get more demand)
# These weights sum to 7 (representing 7 days)
daily_weights = {
    "Mon": 0.8,
    "Tue": 0.8,
    "Wed": 0.9,
    "Thu": 1.0,
    "Fri": 1.3,
    "Sat": 1.6,
    "Sun": 1.6
}

for (i, row) in current.iterrows():
    movie_id = i
    runtimes[movie_id] = row['runtime']
    
    # Get weekly demand per theater and convert to int
    weekly_demand = int(row['weekly_demand_per_theater'])
    
    # Distribute weekly demand across days based on weights
    for day in DAYS:
        # Each day gets its weighted share of weekly demand
        daily_demand = int(weekly_demand * daily_weights[day] / 7.0)
        demand[(movie_id, day)] = daily_demand
    
    movie_ids.append(movie_id)

print(f"Loaded {len(movie_ids)} movies")
print(f"Example demands for movie {movie_ids[0]}:")
for day in DAYS:
    print(f"  {day}: {demand[(movie_ids[0], day)]} tickets")

Loaded 14 movies
Example demands for movie 267:
  Mon: 6 tickets
  Tue: 6 tickets
  Wed: 7 tickets
  Thu: 8 tickets
  Fri: 10 tickets
  Sat: 13 tickets
  Sun: 13 tickets


In [59]:
def createModel(runtimes, demand, ticket_price, max_showings_per_day, operating_min_per_day, screen_capacity, movie_ids, days, screens, buffer_min, verbose=False):
    
    m = gp.Model("CinemaShowtimeScheduling")
    if not verbose:
        m.Params.OutputFlag = 0  # silence output by default
    
    # Decision variables
    x = m.addVars(
        movie_ids, days, screens,
        vtype=GRB.INTEGER,
        lb=0,
        name="x"
    )
    
    # r[i,d] >= 0 continuous - realized tickets
    r = m.addVars(
        movie_ids, days,
        vtype=GRB.CONTINUOUS,
        lb=0.0,
        name="r"
    )
    
    # s[i,d] binary - whether movie is scheduled
    s = m.addVars(
        movie_ids, days,
        vtype=GRB.BINARY,
        name="s"
    )
    
    # Objective: maximize total revenue
    m.setObjective(
        gp.quicksum(ticket_price * r[i, d] for i in movie_ids for d in days),
        GRB.MAXIMIZE
    )
    
    # 1) Screen time constraint per screen & day
    #    sum_i x[i,d,j] * (runtime_i + buffer) <= operating_min_per_day[d]
    for d in days:
        for j in screens:
            m.addConstr(
                gp.quicksum(x[i, d, j] * (runtimes[i] + buffer_min) for i in movie_ids)
                <= operating_min_per_day[d],
                name=f"Time_{d}_{j}"
            )
    
    # 2) Capacity: r[i,d] <= sum_j x[i,d,j] * capacity_j
    for i in movie_ids:
        for d in days:
            m.addConstr(
                r[i, d] <= gp.quicksum(x[i, d, j] * screen_capacity[j] for j in screens),
                name=f"Capacity_{i}_{d}"
            )
    
    # 3) Demand: r[i,d] <= A[i,d]
    for i in movie_ids:
        for d in days:
            demand_id = demand.get((i, d), 0)
            m.addConstr(
                r[i, d] <= demand_id,
                name=f"Demand_{i}_{d}"
            )
    
    # 4) Frequency + schedule linking
    for i in movie_ids:
        for d in days:
            # Frequency limit: total showings per day
            # Link to s: if s[i,d] = 0 => no showings
            m.addConstr(
                gp.quicksum(x[i, d, j] for j in screens)
                <= max_showings_per_day * s[i, d],
                name=f"FreqLimit_{i}_{d}"
            )
    
    # Return model AND variables so we can access them later
    return m, x, r, s

In [60]:
# Create model and get variable references
model, x, r, s = createModel(
    runtimes, demand, TICKET_PRICE, MAX_SHOWINGS_PER_MOVIE_PER_DAY, 
    OPERATING_MIN_PER_DAY, SCREEN_CAPACITIES, movie_ids, DAYS, SCREENS, 
    BUFFER_MIN, verbose=True
)

print(f"\nModel created with {len(movie_ids)} movies, {len(DAYS)} days, {len(SCREENS)} screens")
print(f"Total decision variables: {model.NumVars}")
print(f"Total constraints: {model.NumConstrs}")


Model created with 14 movies, 7 days, 19 screens
Total decision variables: 0
Total constraints: 0


In [61]:
# Test: Run optimization directly to check model
model.optimize()

print(f"\nOptimal objective value: ${model.objVal:,.2f}")
print(f"Number of non-zero variables: {sum(1 for v in model.getVars() if v.X > 1e-6)}")

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.2.0 24C2101)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 427 rows, 2058 columns and 5880 nonzeros (Max)
Model fingerprint: 0xa9db8c81
Model has 98 linear objective coefficients
Variable types: 98 continuous, 1960 integer (98 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 4e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 390 rows and 1768 columns
Presolve time: 0.03s
Presolved: 37 rows, 290 columns, 632 nonzeros
Found heuristic solution: objective 197789.28000
Variable types: 20 continuous, 270 integer (2 binary)

Root relaxation: objective 3.175622e+05, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth I

In [62]:
def optimize_showtimes(model, x, r, s, movie_ids, days, screens, verbose=False):
    """
    Optimize the showtime scheduling model and extract results
    
    Args:
        model: Gurobi model
        x: Decision variables for showings per movie/day/screen
        r: Decision variables for realized tickets
        s: Binary variables for whether movie is scheduled
        movie_ids: List of movie IDs
        days: List of days
        screens: List of screen names
        verbose: Whether to print status
    """
    
    model.optimize()
    
    status_code = model.Status
    if status_code == GRB.OPTIMAL:
        status = "Optimal"
    elif status_code == GRB.INFEASIBLE:
        status = "Infeasible"
    elif status_code == GRB.UNBOUNDED:
        status = "Unbounded"
    else:
        status = f"Status_{status_code}"
    
    total_revenue = model.objVal if status_code == GRB.OPTIMAL else None
    
    if verbose:
        print("Status:", status)
        print(f"Total revenue: ${total_revenue:,.2f}" if total_revenue else "N/A")
    
    schedule_rows = []
    if status_code == GRB.OPTIMAL:
        for i in movie_ids:
            for d in days:
                for j in screens:
                    val = x[i, d, j].X
                    if val > 1e-6:
                        schedule_rows.append({
                            "movie_id": i,
                            "day": d,
                            "screen": j,
                            "showings": int(round(val))
                        })
    
    schedule_df = pd.DataFrame(schedule_rows)
    
    realized_rows = []
    if status_code == GRB.OPTIMAL:
        for i in movie_ids:
            for d in days:
                realized_rows.append({
                    "movie_id": i,
                    "day": d,
                    "realized_tickets": r[i, d].X,
                    "scheduled_flag": int(round(s[i, d].X))
                })
    realized_df = pd.DataFrame(realized_rows)
    
    return {
        "status": status,
        "total_revenue": total_revenue,
        "schedule_df": schedule_df,
        "realized_df": realized_df
    }

In [63]:
# Optimize and get results
results = optimize_showtimes(model, x, r, s, movie_ids, DAYS, SCREENS, verbose=True)

print(f"\n{'='*60}")
print("OPTIMIZATION RESULTS")
print(f"{'='*60}")
print(f"Status: {results['status']}")
print(f"Total Revenue: ${results['total_revenue']:,.2f}" if results['total_revenue'] else "N/A")
print(f"\nSchedule has {len(results['schedule_df'])} showtime slots")
print(f"Movies scheduled: {results['realized_df'][results['realized_df']['scheduled_flag'] == 1].shape[0]} movie-day combinations")

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[rosetta2] - Darwin 24.2.0 24C2101)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 427 rows, 2058 columns and 5880 nonzeros (Max)
Model fingerprint: 0xa9db8c81
Model has 98 linear objective coefficients
Variable types: 98 continuous, 1960 integer (98 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 4e+03]
Presolved: 37 rows, 290 columns, 632 nonzeros

Continuing optimization...


Explored 1 nodes (20 simplex iterations) in 0.05 seconds (0.01 work units)
Most recent optimization runtime was 0.01 seconds (0.00 work units)
Thread count was 10 (of 10 available processors)

Solution count 3: 317562 197789 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.175621800000e+05, best bound 3.175621800000e+05, gap 0.0000%
Status: Optimal
Total 

In [64]:
# Analyze the schedule
print("\n" + "="*60)
print("SCHEDULE ANALYSIS")
print("="*60)

if len(results['schedule_df']) > 0:
    schedule_df = results['schedule_df']
    realized_df = results['realized_df']
    
    # Showings per movie
    print("\nShowings per movie:")
    showings_per_movie = schedule_df.groupby('movie_id')['showings'].sum().sort_values(ascending=False)
    for movie_id, count in showings_per_movie.items():
        movie_title = current.loc[movie_id, 'title'] if movie_id in current.index else f"Movie {movie_id}"
        print(f"  {movie_title}: {count} showings")
    
    # Tickets sold per movie
    print("\nTickets sold per movie:")
    tickets_per_movie = realized_df[realized_df['scheduled_flag'] == 1].groupby('movie_id')['realized_tickets'].sum().sort_values(ascending=False)
    for movie_id, tickets in tickets_per_movie.items():
        movie_title = current.loc[movie_id, 'title'] if movie_id in current.index else f"Movie {movie_id}"
        revenue = tickets * TICKET_PRICE
        print(f"  {movie_title}: {tickets:,.0f} tickets (${revenue:,.2f})")
    
    # Showings per day
    print("\nShowings per day:")
    showings_per_day = schedule_df.groupby('day')['showings'].sum()
    for day in DAYS:
        count = showings_per_day.get(day, 0)
        print(f"  {day}: {count} showings")
    
    # Screen utilization
    print("\nScreen utilization:")
    screens_used = schedule_df.groupby('screen')['showings'].sum().sort_values(ascending=False)
    print(f"  Screens used: {len(screens_used)} / {len(SCREENS)}")
    for screen, count in screens_used.head(10).items():
        print(f"  {screen}: {count} showings")
    
    # Display sample schedule
    print("\n" + "="*60)
    print("SAMPLE SCHEDULE (first 20 entries)")
    print("="*60)
    display(schedule_df.head(20))
    
else:
    print("\n⚠️ No schedule generated - check if demand is too low or constraints are too tight")


SCHEDULE ANALYSIS

Showings per movie:
  The Smashing Machine: 30 showings
  Kiss of the Spider Woman: 11 showings
  Shelby Oaks: 9 showings
  Truth & Treason: 9 showings
  Anniversary: 7 showings
  Black Phone 2: 7 showings
  Good Fortune: 7 showings
  Last Days: 7 showings
  Re-Election: 7 showings
  Regretting You: 7 showings
  Roofman: 7 showings
  Soul on Fire: 7 showings
  Stitch Head: 7 showings
  The Mastermind: 7 showings

Tickets sold per movie:
  The Smashing Machine: 16,619 tickets ($187,960.89)
  Kiss of the Spider Woman: 3,723 tickets ($42,107.13)
  Truth & Treason: 1,123 tickets ($12,701.13)
  Shelby Oaks: 1,108 tickets ($12,531.48)
  Roofman: 1,033 tickets ($11,683.23)
  Re-Election: 969 tickets ($10,959.39)
  Black Phone 2: 803 tickets ($9,081.93)
  Good Fortune: 729 tickets ($8,244.99)
  Soul on Fire: 670 tickets ($7,577.70)
  Regretting You: 610 tickets ($6,899.10)
  Stitch Head: 244 tickets ($2,759.64)
  The Mastermind: 240 tickets ($2,714.40)
  Last Days: 144 tick

,movie_id,day,screen,showings
0,267,Mon,Screen_18,1
1,267,Tue,Screen_10,1
2,267,Wed,Screen_19,1
3,267,Thu,Screen_8,1
4,267,Fri,Screen_10,1
5,267,Sat,Screen_1,1
6,267,Sun,Screen_17,1
7,475,Mon,Screen_13,1
8,475,Tue,Screen_13,1
9,475,Wed,Screen_12,1


## Export Results for Sensitivity Analysis

In [159]:
# Create results directory if it doesn't exist
results_dir = Path("results")
results_dir.mkdir(exist_ok=True)

# Export schedule
schedule_file = results_dir / "baseline_schedule.csv"
results['schedule_df'].to_csv(schedule_file, index=False)
print(f"✓ Schedule exported to: {schedule_file}")

# Export realized tickets/revenue
realized_file = results_dir / "baseline_realized.csv"
results['realized_df'].to_csv(realized_file, index=False)
print(f"✓ Realized tickets exported to: {realized_file}")

# Export summary statistics
summary_data = {
    'metric': [
        'total_revenue',
        'total_showings',
        'total_tickets_sold',
        'num_movies_scheduled',
        'num_screens_used',
        'avg_showings_per_movie',
        'avg_tickets_per_showing',
        'capacity_utilization'
    ],
    'value': [
        results['total_revenue'],
        len(results['schedule_df']),
        results['realized_df']['realized_tickets'].sum(),
        results['realized_df']['scheduled_flag'].sum(),
        results['schedule_df']['screen'].nunique(),
        results['schedule_df'].groupby('movie_id')['showings'].sum().mean(),
        results['realized_df']['realized_tickets'].sum() / results['schedule_df']['showings'].sum() if len(results['schedule_df']) > 0 else 0,
        results['realized_df']['realized_tickets'].sum() / (results['schedule_df']['showings'].sum() * 210) if len(results['schedule_df']) > 0 else 0  # assuming avg capacity 210
    ]
}
summary_df = pd.DataFrame(summary_data)
summary_file = results_dir / "baseline_summary.csv"
summary_df.to_csv(summary_file, index=False)
print(f"✓ Summary statistics exported to: {summary_file}")

# Export parameters used
params_data = {
    'parameter': [
        'ticket_price',
        'buffer_min',
        'max_showings_per_movie_per_day',
        'num_screens',
        'operating_hours_per_day',
        'num_movies',
        'num_days'
    ],
    'value': [
        TICKET_PRICE,
        BUFFER_MIN,
        MAX_SHOWINGS_PER_MOVIE_PER_DAY,
        NUM_SCREENS,
        OPERATING_MIN_PER_DAY['Mon'] / 60,  # convert to hours
        len(movie_ids),
        len(DAYS)
    ]
}
params_df = pd.DataFrame(params_data)
params_file = results_dir / "baseline_parameters.csv"
params_df.to_csv(params_file, index=False)
print(f"✓ Parameters exported to: {params_file}")

# Export movie data
movie_export = current[['title', 'runtime', 'weekly_demand_per_theater']].copy()
movie_export['movie_id'] = current.index
movie_file = results_dir / "movie_data.csv"
movie_export.to_csv(movie_file, index=False)
print(f"✓ Movie data exported to: {movie_file}")

# Export demand data (for sensitivity analysis)
demand_data = []
for (movie_id, day), demand_val in demand.items():
    demand_data.append({
        'movie_id': movie_id,
        'day': day,
        'demand': demand_val
    })
demand_df = pd.DataFrame(demand_data)
demand_file = results_dir / "demand_data.csv"
demand_df.to_csv(demand_file, index=False)
print(f"✓ Demand data exported to: {demand_file}")

print(f"\n{'='*60}")
print("All baseline results exported successfully!")
print(f"{'='*60}")

✓ Schedule exported to: results/baseline_schedule.csv
✓ Realized tickets exported to: results/baseline_realized.csv
✓ Summary statistics exported to: results/baseline_summary.csv
✓ Parameters exported to: results/baseline_parameters.csv
✓ Movie data exported to: results/movie_data.csv
✓ Demand data exported to: results/demand_data.csv

All baseline results exported successfully!
